In [1]:
using ITensors

# Basic DMRG

In [3]:
# Define the number of sites
N = 100
sites = siteinds("S=1/2",N)
@show sites[1]

sites[1] = (dim=2|id=50|"S=1/2,Site,n=1")


(dim=2|id=50|"S=1/2,Site,n=1")

In [4]:
# Define Hamiltonian (nearest site spin-spin interaction)
os = OpSum() 
for j=1:N-1 
    os += "Sz",j,"Sz",j+1
    os += 1/2,"S+",j,"S-",j+1 
    os += 1/2,"S-",j,"S+",j+1 
end

# Generate an instance of this Hamiltonian
H = MPO(os,sites);

# Define the ansatz state
psi0 = randomMPS(sites; linkdims=10);

In [5]:
# Define the number of sweeps
sweeps = Sweeps(6) 

# Set the maximum bond dimension in each sweep
setmaxdim!(sweeps, 10,20,100,100,200,300) 

# Set the maximum truncation error 
setcutoff!(sweeps, 1E-11)

In [6]:
# Solve for ground state
energy, psi = dmrg(H,psi0, sweeps) 
println("G.S. energy = $energy")

After sweep 1 energy=-44.07669935740897  maxlinkdim=10 maxerr=2.91E-03 time=12.243
After sweep 2 energy=-44.121721339320274  maxlinkdim=20 maxerr=1.52E-06 time=0.155
After sweep 3 energy=-44.127567249572564  maxlinkdim=76 maxerr=9.94E-12 time=0.701
After sweep 4 energy=-44.127734900930115  maxlinkdim=100 maxerr=4.16E-11 time=2.208
After sweep 5 energy=-44.127739849763  maxlinkdim=132 maxerr=1.00E-11 time=3.071
After sweep 6 energy=-44.127739890041276  maxlinkdim=146 maxerr=1.00E-11 time=4.620
G.S. energy = -44.127739890041276


# Quantum Number Conserving DMRG

In [7]:
# Define sites, with conservation of quantum numbers turned on
N = 100
sites = siteinds("S=1",N;conserve_qns=true);

In [8]:
# Define Hamiltonian
os = OpSum()
for j=1:N-1
  os += "Sz",j,"Sz",j+1
  os += 1/2,"S+",j,"S-",j+1
  os += 1/2,"S-",j,"S+",j+1
end
H = MPO(os,sites);

In [9]:
# Define initial state, we have a alternating product state of up and down such that the total Sz=0
state = [isodd(n) ? "Up" : "Dn" for n=1:N]
psi0 = MPS(sites,state)
@show flux(psi0)

flux(psi0) = QN("Sz",0)


QN("Sz",0)

In [10]:
# Set the maximum number of sweeps
nsweeps = 5

# Set the maximum bond dimension of each sweep
maxdim = [10,20,100,100,200]

# Set the maximum truncation error
cutoff = [1E-10]

# Solve for the GS energy
energy, psi = dmrg(H,psi0; nsweeps, maxdim, cutoff)


After sweep 1 energy=-138.32393649712677  maxlinkdim=9 maxerr=1.12E-15 time=21.429
After sweep 2 energy=-138.93726652331463  maxlinkdim=20 maxerr=3.83E-06 time=1.303
After sweep 3 energy=-138.94008403991265  maxlinkdim=91 maxerr=1.00E-10 time=1.800
After sweep 4 energy=-138.94008607738797  maxlinkdim=100 maxerr=9.99E-11 time=2.663
After sweep 5 energy=-138.9400860761833  maxlinkdim=95 maxerr=9.98E-11 time=2.492


(-138.9400860761833, MPS
[1] ((dim=3|id=399|"Link,l=1") <Out>
 1: QN("Sz",-2) => 1
 2: QN("Sz",0) => 1
 3: QN("Sz",2) => 1, (dim=3|id=583|"S=1,Site,n=1") <Out>
 1: QN("Sz",2) => 1
 2: QN("Sz",0) => 1
 3: QN("Sz",-2) => 1)
[2] ((dim=9|id=148|"Link,l=2") <Out>
 1: QN("Sz",-4) => 1
 2: QN("Sz",-2) => 2
 3: QN("Sz",0) => 3
 4: QN("Sz",2) => 2
 5: QN("Sz",4) => 1, (dim=3|id=228|"S=1,Site,n=2") <Out>
 1: QN("Sz",2) => 1
 2: QN("Sz",0) => 1
 3: QN("Sz",-2) => 1, (dim=3|id=399|"Link,l=1") <In>
 1: QN("Sz",-2) => 1
 2: QN("Sz",0) => 1
 3: QN("Sz",2) => 1)
[3] ((dim=3|id=117|"S=1,Site,n=3") <Out>
 1: QN("Sz",2) => 1
 2: QN("Sz",0) => 1
 3: QN("Sz",-2) => 1, (dim=27|id=566|"Link,l=3") <Out>
 1: QN("Sz",-6) => 1
 2: QN("Sz",-4) => 3
 3: QN("Sz",-2) => 6
 4: QN("Sz",0) => 7
 5: QN("Sz",2) => 6
 6: QN("Sz",4) => 3
 7: QN("Sz",6) => 1, (dim=9|id=148|"Link,l=2") <In>
 1: QN("Sz",-4) => 1
 2: QN("Sz",-2) => 2
 3: QN("Sz",0) => 3
 4: QN("Sz",2) => 2
 5: QN("Sz",4) => 1)
[4] ((dim=3|id=626|"S=1,Site,n=4"

In [11]:
# Verify the conservation of Sz
@show flux(psi)

flux(psi) = QN("Sz",0)


QN("Sz",0)

# AKLT Ground State Simulation

In [23]:
# Define the number of sites
N = 30
sites = siteinds("S=1",N);

In [24]:
# Define Hamiltonian (nearest site spin-spin interaction)
os = OpSum() 
for j=1:N-1 
    # S*S Terms
    # Sz Sz
    os += "Sz", j, "Sz", j+1

    # S+ S- and S- S+
    os += 1/2, "S+", j, "S-", j+1
    os += 1/2, "S-", j, "S+", j+1

    # Square terms in (S*S)^2
    # (1/3) * (Sz Sz)^2 term
    os += (1/3), "Sz", j, "Sz", j+1, "Sz", j, "Sz", j+1
    # (1/3) * (1/4) (S+ S- S+ S-) term
    os += (1/3) * (1/4), "S+", j, "S-", j+1, "S+", j, "S-", j+1
    os += (1/3) * (1/4), "S-", j, "S+", j+1, "S-", j, "S+", j+1

    # Cross terms in (S*S)^2
    # (1/3) * (1/2) Sz Sz S+ S- terms
    os += (1/3) * (1/2), "Sz", j, "Sz", j+1, "S+", j, "S-", j+1
    os += (1/3) * (1/2), "S+", j, "S-", j+1, "Sz", j, "Sz", j+1

    # (1/3) * (1/2) Sz Sz S- S+ terms
    os += (1/3) * (1/2), "Sz", j, "Sz", j+1, "S-", j, "S+", j+1
    os += (1/3) * (1/2), "S-", j, "S+", j+1, "Sz", j, "Sz", j+1

    # (1/3) * (1/4) S+ S- S- S+ terms
    os += (1/3) * (1/4), "S+", j, "S-", j+1, "S-", j, "S+", j+1
    os += (1/3) * (1/4), "S-", j, "S+", j+1, "S+", j, "S-", j+1
end

# Convert the OpSum to an MPO (Matrix Product Operator)
H_AKLT = MPO(os, sites);

In [25]:
# Define the ansatz state
psi0 = randomMPS(sites; linkdims=10);

In [26]:
# Set the maximum number of sweeps
nsweeps = 5

# Set the maximum bond dimension of each sweep
maxdim = [10,20,100,100,200]

# Set the maximum truncation error
cutoff = [1E-10]

# Solve for the GS energy
temp_energy, psi = dmrg(H_AKLT,psi0; nsweeps, maxdim, cutoff);
norm = inner(psi, psi);
energy = inner(psi', H_AKLT, psi)/norm
println("The ground state energy is", energy)

After sweep 1 energy=-32.648159947767454  maxlinkdim=10 maxerr=1.02E-02 time=0.135
After sweep 2 energy=-32.66652506880712  maxlinkdim=20 maxerr=1.57E-08 time=0.297
After sweep 3 energy=-32.66666484911889  maxlinkdim=12 maxerr=1.00E-10 time=0.156
After sweep 4 energy=-32.66666666506971  maxlinkdim=4 maxerr=9.70E-11 time=0.050
After sweep 5 energy=-32.66666666666668  maxlinkdim=2 maxerr=1.40E-12 time=0.039
The ground state energy is-32.66666666666666


In [27]:
# Calculate the correlation of some operator between the ith and jth site
function Corr_ij(psi::MPS, operator1::String, operator2::String, i::Int64, j::Int64)
    # Get the number of sites from the MPS
    N = length(siteinds(psi))

    # Check if indices are within the valid range
    if i < 1 || i > N || j < 1 || j > N
        @warn "Indices i=$i and j=$j are out of bounds for the MPS with $N sites"
        return nothing
    end

    # Calculate the full correlation matrix
    Corr = correlation_matrix(psi, operator1, operator2)

    # Extract the corresponding element of correlation matrix
    return Corr[i, j]
end


Corr_ij (generic function with 1 method)

In [29]:
Corr_ij(psi, "Sz", "Sz", 4, 2)

0.1481481481779856

# AKLT Exact Ground State

In [2]:
# Define the number of sites
N = 4

# Create physical indices for each site with dimension 3
phys_indices = [Index(3, "S=1, Site, n=$n") for n in 1:N]
#phys_indices = siteinds("S=1",N);

# Create bond indices with dimension 2 for a 1D chain
bond_indices = [Index(2, "Bond, n=$n") for n in 1:N]  # +1 to handle both ends

# Initialize a vector to store the MPS tensors
tensors = Vector{ITensor}(undef, N);

In [3]:
# Calculate the normalization factor
norm_factor = 2/sqrt(3)

# Define matrices corresponding to each physical index value, normalized
matrices = [ [0 1/sqrt(2) * norm_factor; 0 0],  # Matrix when physical index is 1, A+
             [-1/2 * norm_factor 0; 0 1/2 * norm_factor],  # Matrix when physical index is 2, A0
             [0 0; -1/sqrt(2) * norm_factor 0] ]  # Matrix when physical index is 3, A-

# To verify, let's print out the matrices
for (idx, mat) in enumerate(matrices)
    println("Matrix for physical index ", idx, ":")
    println(mat)
    println()  # Add an empty line for better readability
end

Matrix for physical index 1:
[0.0 0.816496580927726; 0.0 0.0]

Matrix for physical index 2:
[-0.5773502691896258 0.0; 0.0 0.5773502691896258]

Matrix for physical index 3:
[0.0 0.0; -0.816496580927726 0.0]



In [19]:
# Assign matrix element to bulk sites
# Loop over each site
for n in 1:N
    tensors[n] = ITensor(bond_indices[(n-1)%N+1], phys_indices[n], bond_indices[(n)%N+1])
    #@show n, tensors[n]

    # Loop over physical index
    for p in 1:3
        
        # Loop over each element in the matrix
        for idx in CartesianIndices(matrices[p])
            # Access matrix element using idx and assign it to the tensor
            # Note the correction in the physical index access: should be phys_indices[n]
            # Tuple(idx)... splits the CartesianIndex into its component parts
            i, j = Tuple(idx)
            tensors[n][bond_indices[(n-1)%N+1] => i, phys_indices[n] => p, bond_indices[(n)%N+1] => j] = matrices[p][idx];
        end
    end
end

In [20]:
psi_exact = MPS(tensors);
#@show siteinds(psi_exact)

MPS
[1] ((dim=2|id=300|"Bond,n=1"), (dim=3|id=614|"S=1,Site,n=1"), (dim=2|id=869|"Bond,n=2"))
[2] ((dim=2|id=869|"Bond,n=2"), (dim=3|id=236|"S=1,Site,n=2"), (dim=2|id=950|"Bond,n=3"))
[3] ((dim=2|id=950|"Bond,n=3"), (dim=3|id=844|"S=1,Site,n=3"), (dim=2|id=889|"Bond,n=4"))
[4] ((dim=2|id=889|"Bond,n=4"), (dim=3|id=120|"S=1,Site,n=4"), (dim=2|id=300|"Bond,n=1"))


In [22]:
# Define Hamiltonian (nearest site spin-spin interaction)
os = OpSum() 
for j=1:N-1 
    # S*S Terms
    # Sz Sz
    os += "Sz", j, "Sz", j+1

    # S+ S- and S- S+
    os += 1/2, "S+", j, "S-", j+1
    os += 1/2, "S-", j, "S+", j+1

    # Square terms in (S*S)^2
    # (1/3) * (Sz Sz)^2 term
    os += (1/3), "Sz", j, "Sz", j+1, "Sz", j, "Sz", j+1
    # (1/3) * (1/4) (S+ S- S+ S-) term
    os += (1/3) * (1/4), "S+", j, "S-", j+1, "S+", j, "S-", j+1
    os += (1/3) * (1/4), "S-", j, "S+", j+1, "S-", j, "S+", j+1

    # Cross terms in (S*S)^2
    # (1/3) * (1/2) Sz Sz S+ S- terms
    os += (1/3) * (1/2), "Sz", j, "Sz", j+1, "S+", j, "S-", j+1
    os += (1/3) * (1/2), "S+", j, "S-", j+1, "Sz", j, "Sz", j+1

    # (1/3) * (1/2) Sz Sz S- S+ terms
    os += (1/3) * (1/2), "Sz", j, "Sz", j+1, "S-", j, "S+", j+1
    os += (1/3) * (1/2), "S-", j, "S+", j+1, "Sz", j, "Sz", j+1

    # (1/3) * (1/4) S+ S- S- S+ terms
    os += (1/3) * (1/4), "S+", j, "S-", j+1, "S-", j, "S+", j+1
    os += (1/3) * (1/4), "S-", j, "S+", j+1, "S+", j, "S-", j+1
end

# Convert the OpSum to an MPO (Matrix Product Operator)
H_AKLT = MPO(os, phys_indices)

MPO
[1] ((dim=14|id=999|"Link,l=1"), (dim=3|id=614|"S=1,Site,n=1")', (dim=3|id=614|"S=1,Site,n=1"))
[2] ((dim=14|id=999|"Link,l=1"), (dim=14|id=45|"Link,l=2"), (dim=3|id=236|"S=1,Site,n=2")', (dim=3|id=236|"S=1,Site,n=2"))
[3] ((dim=14|id=45|"Link,l=2"), (dim=14|id=594|"Link,l=3"), (dim=3|id=844|"S=1,Site,n=3")', (dim=3|id=844|"S=1,Site,n=3"))
[4] ((dim=14|id=594|"Link,l=3"), (dim=3|id=120|"S=1,Site,n=4")', (dim=3|id=120|"S=1,Site,n=4"))


In [29]:
# Contract MPS with MPO to get the expectation value
#@show psi_exact
#@show H_AKLT
temp_psi = apply(H_AKLT, psi_exact)
psi2 = temp_psi[1]*temp_psi[2]*temp_psi[3]*temp_psi[4]
psi1 =  psi_exact[1]*psi_exact[2]*psi_exact[3]*psi_exact[4]
#H = H_AKLT[1]*H_AKLT[2]*H_AKLT[3]*H_AKLT[4]
#@show psi1, psi2
@show psi1*psi2
#expectation_value = inner(psi_exact', H_AKLT, psi_exact);
#expectation_value = inner(psi_exact, apply(H_AKLT, psi_exact))
#println("Expectation value: ", expectation_value)

psi1 * psi2 = ITensor ord=0
NDTensors.Dense{Float64, Vector{Float64}}
 0-dimensional
-2.074074074074074


ITensor ord=0
NDTensors.Dense{Float64, Vector{Float64}}